<a href="https://colab.research.google.com/github/Danicoder/Especializacion_IA_BigData/blob/main/Ejercicio_Pr%C3%A1ctico_19_Hadoop_Map_Reduce_Streaming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# I: Instalación de Hadoop en el entorno

1. Descargamos Hadoop.

In [1]:
!wget https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz


--2025-03-05 12:15:34--  https://downloads.apache.org/hadoop/common/hadoop-3.3.6/hadoop-3.3.6.tar.gz
Resolving downloads.apache.org (downloads.apache.org)... 135.181.214.104, 88.99.208.237, 2a01:4f9:3a:2c57::2, ...
Connecting to downloads.apache.org (downloads.apache.org)|135.181.214.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 730107476 (696M) [application/x-gzip]
Saving to: ‘hadoop-3.3.6.tar.gz’

hadoop-3.3.6.tar.gz 100%[===================>] 696.28M  25.4MB/s    in 28s     

2025-03-05 12:16:02 (25.0 MB/s) - ‘hadoop-3.3.6.tar.gz’ saved [730107476/730107476]



2. Descompresión de la distribución de Hadoop

In [2]:
!tar -xzf hadoop-3.3.6.tar.gz

3. Copiar Hadoop a /usr/local

In [3]:
!cp -r hadoop-3.3.6/ /usr/local/

## II: Configurar el Hadoop JAVA HOME

Hadoop requiere que se establezca la ruta de acceso a Java, ya sea como una variable de entorno o en el archivo de configuración de Hadoop.

4. Buscamos cual es la dirección de Java en la máquina Google Colab

In [4]:
!readlink -f /usr/bin/java | sed "s:bin/java::"

/usr/lib/jvm/java-11-openjdk-amd64/


5. Establecemos mediante código Python el valor de esta variable y actualizamos el *PATH* del sistema.



In [5]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64/"
os.environ["PATH"] = os.environ["PATH"] + ":" + "/usr/local/hadoop-3.3.6/bin"

# III:  Ejecutando Hadoop y Descargar Datos Patentes USA 1975-1999

6. Compruebo que se está ejecutando Hadoop en el sistema.

In [6]:
#Running Hadoop
!/usr/local/hadoop-3.3.6/bin/hadoop

Usage: hadoop [OPTIONS] SUBCOMMAND [SUBCOMMAND OPTIONS]
 or    hadoop [OPTIONS] CLASSNAME [CLASSNAME OPTIONS]
  where CLASSNAME is a user-provided Java class

  OPTIONS is none or any of:

buildpaths                       attempt to add class files from build tree
--config dir                     Hadoop config directory
--debug                          turn on shell script debug mode
--help                           usage information
hostnames list[,of,host,names]   hosts to use in worker mode
hosts filename                   list of hosts to use in worker mode
loglevel level                   set the log4j level for this command
workers                          turn on worker mode

  SUBCOMMAND is one of:


    Admin Commands:

daemonlog     get/set the log level for each daemon

    Client Commands:

archive       create a Hadoop archive
checknative   check native Hadoop and compression libraries availability
classpath     prints the class path needed to get the Hadoop jar and the re

7. Descargo los datos de patentes de USA desde 1975 hasta 1999, lo descomprimo y  mostramos cabecera de fichero resultante.

In [7]:
!wget https://data.nber.org/patents/acite75_99.zip
!unzip acite75_99.zip
!head cite75_99.txt

--2025-03-05 12:16:22--  https://data.nber.org/patents/acite75_99.zip
Resolving data.nber.org (data.nber.org)... 198.71.6.39, 198.71.6.22
Connecting to data.nber.org (data.nber.org)|198.71.6.39|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82415288 (79M) [application/zip]
Saving to: ‘acite75_99.zip’

acite75_99.zip      100%[===================>]  78.60M  25.7MB/s    in 3.2s    

2025-03-05 12:16:26 (24.6 MB/s) - ‘acite75_99.zip’ saved [82415288/82415288]

Archive:  acite75_99.zip
  inflating: cite75_99.txt           
"CITING","CITED"
3858241,956203
3858241,1324234
3858241,3398406
3858241,3557384
3858241,3634889
3858242,1515701
3858242,3319261
3858242,3668705
3858242,3707004


#IV: MapReduce en Python

El API de *MapReduce* está programado en *Java*, pero mediante *Hadoop Streaming* permite crear y ejecutar trabajos Map/Reduce con cualquier ejecutable o script como mapeador y/o reductor.

En los ejercicios que aparecen a continuación usaremos el sistema de tuberías Unix `|`

Mas información sobre Map/Reduce en el siguiente [enlace](https://hadoop.apache.org/docs/current/hadoop-mapreduce-client/hadoop-mapreduce-client-core/MapReduceTutorial.html)

## Mapper
El mapeador Divide cada línea del archivo de entrada en campos utilizando la coma como delimitador.


Devuelve el segundo campo como clave y el primer campo como valor.

El código siguiente **NO es necesario ejecutarlo**, lo tienes en los recursos de *Aules* del ejercicio práctico con el nombre **mapper.py**. Simplemente súbelo dentro de la carpeta `/content/`.

In [8]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

import sys

# Procesa las entradas y genera pares clave-valor.
# Divide cada línea del archivo de entrada en campos utilizando la coma como delimitador.
# Emite el segundo campo como clave y el primer campo como valor.
for line in sys.stdin:
    citation = line.strip().split(",")
    if len(citation) >= 2:
        print("%s\t%s" % (citation[1], citation[0]))

## Reducer
El *reducer* recibimos la salida del *mapper* y realizamos las siguientes acciones:
- Lee pares clave-valor de la entrada estándar.
- Combina todos los valores asociados con la misma clave en una lista.
- Luego, emite la clave junto con esta lista de valores.

El código siguiente **NO es necesario ejecutarlo**, lo tienes en los recursos de *Aules* del ejercicio práctico.con el nombre **reducer.py**. Simplemente súbelo dentro de la carpeta `/content/` utilzando la opción de *Subir* en el menú `Archivos`.

In [9]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-


import sys

current_key = None
values = []

# Lee pares clave-valor de la entrada estándar.
# Combina todos los valores asociados con la misma clave en una lista.
# Luego, emite la clave junto con esta lista de valores.
for line in sys.stdin:
    key, value = line.strip().split("\t", 1)

    if current_key == key:
        values.append(value)
    else:
        if current_key:
            print("%s\t%s" % (current_key, ",".join(values)))
        current_key = key
        values = [value]

if current_key:
    print("%s\t%s" % (current_key, ",".join(values)))

#V: Hadoop Streaming

Una vez comprobados que los algoritmos de *mapeo* y *reducción* funcionan, los procesaremos dentro de `Hadoop` para aprovechar la computación distribuida.

Emplearemos **Hadoop Streaming**, que permite ejecutar Map/Reduce codificados en cualquier lenguaje de programación que pueda leer de la entrada estándar (stdin) y escribir a la salida estándar (stdout). De este manera, Hadoop Streaming envía los datos en crudo al mapper vía stdin y tras procesarlos, se los pasa al reducer vía stdout.


La sintaxis para ejecutar los jobs es:

`mapred streaming -input EntradaHDFS -output SalidaHDFS -mapper scriptMapper   -reducer scriptReducer`



8. Tenemos que dar permisos de ejecución a ambos scripts para que *Hadoop Streaming* los pueda procesar.

In [38]:
!mkdir /content/EntradaHDFS

In [29]:
!chmod +x mapper.py reducer.py

In [39]:
!mapred streaming -input EntradaHDFS -output SalidaHDFS -mapper mapper.py -reducer reducer.py

2025-03-05 14:57:35,199 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-05 14:57:35,390 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-05 14:57:35,390 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-05 14:57:35,425 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-05 14:57:35,695 INFO mapred.FileInputFormat: Total input files to process : 0
2025-03-05 14:57:35,727 INFO mapreduce.JobSubmitter: number of splits:0
2025-03-05 14:57:35,982 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local503843238_0001
2025-03-05 14:57:35,982 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-05 14:57:36,191 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-03-05 14:57:36,193 INFO mapreduce.Job: Running job: job_local503843238_0001
2025-03-05 14:57:36,204 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-03-0

9. Crear un **Map Reduce** con el fichero de **entrada** `cite75_99.txt`, descargando la **salida** en la ruta `/content/salida`. Usar los ficheros `mapper.py` para **mapear** y `reducer.py` como **reductor**. Puesto que vamos a usar los ficheros que tenemos en local, hemos de añadir el parámetro `-file`

**ATENCIÓN:** Recuerda que tienes que poner `!` o `%%bash` delante de los comandos a ejecutar.

In [42]:
!mapred streaming -input /content/cite75_99.txt -output /content/salida -mapper mapper.py -reducer reducer.py -file /content/mapper.py -file /content/reducer.py

2025-03-05 15:05:29,894 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/mapper.py, /content/reducer.py] [] /tmp/streamjob11406460168428958924.jar tmpDir=null
2025-03-05 15:05:30,515 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-05 15:05:30,663 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-05 15:05:30,664 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-05 15:05:30,679 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-05 15:05:30,895 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-05 15:05:30,918 INFO mapreduce.JobSubmitter: number of splits:8
2025-03-05 15:05:31,152 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1031425423_0001
2025-03-05 15:05:31,152 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-05 15:05:31,553 INFO mapred.Lo

10. Si compruebo el directorio  `salida` veo que contiene dos archivos:
- El primero **_SUCCESS** indica que el job de *MapReduce* se ha ejecutado correctamente.
- El segundo `part-00000` contiene el resutlado del mismo.

In [45]:
!ls -l /content/salida

total 154380
-rw-r--r-- 1 root root 158078539 Mar  5 15:06 part-00000
-rw-r--r-- 1 root root         0 Mar  5 15:06 _SUCCESS


11. Muestro el contenido o la cabecera del archivo generado (lo que prefieras).

In [48]:
!head /content/salida/part-00000

"CITED"	"CITING"
1	3964859,4647229
10000	4539112
100000	5031388
1000006	4714284
1000007	4766693
1000011	5033339
1000017	3908629
1000026	4043055
1000033	4190903,4975983


#VI: Hadoop Streaming con comandos Shell de Linux

Vamos a usar comandos de la **Shell de Linux** para hacer de *Mapper* y de *Reducer*, el objetivo que perseguimos es simular el proceso **Map Reduce** con los programas que hemos ejecutado en el apartado anterior con **Hadoop Streaming**.

La sintaxis para ejecutar los jobs es:

`mapred streaming -input EntradaHDFS -output SalidaHDFS -mapper scriptMapper   -reducer scriptReducer`

12. En el *mapeo* extraemos sólo los datos del campo 2 con el comando `cut` y eliminamos duplicados con el *reducer* mediante el comando `uniq`

In [10]:
!mapred streaming -input /content/cite75_99.txt -output /content/apart_12 -mapper 'cut -f 2 -d ,' -reducer 'uniq'

2025-03-05 12:16:30,547 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-05 12:16:30,721 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-05 12:16:30,722 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-05 12:16:30,735 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-05 12:16:30,979 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-05 12:16:31,005 INFO mapreduce.JobSubmitter: number of splits:8
2025-03-05 12:16:31,286 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1004124041_0001
2025-03-05 12:16:31,286 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-05 12:16:31,467 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-03-05 12:16:31,468 INFO mapreduce.Job: Running job: job_local1004124041_0001
2025-03-05 12:16:31,478 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-03

13. Compruebo el directorio de salida `apart_12` y el contenido o la cabecera del archivo resultante.

In [49]:
!ls -l /content/apart_12/

total 28476
-rw-r--r-- 1 root root 29158010 Mar  5 12:17 part-00000
-rw-r--r-- 1 root root        0 Mar  5 12:17 _SUCCESS


14. En este apartado sólo queremos contar ocurrencias. Por tanto no necesitamos el *reductor* y le indicamos que sólo queremos *mapear* con el comando `wc -l`

In [11]:
!mapred streaming -input /content/cite75_99.txt -output /content/apart_14 -mapper 'wc -l'

2025-03-05 12:17:09,061 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-05 12:17:09,195 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-05 12:17:09,195 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-05 12:17:09,224 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-05 12:17:09,401 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-05 12:17:09,425 INFO mapreduce.JobSubmitter: number of splits:8
2025-03-05 12:17:09,622 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local212957500_0001
2025-03-05 12:17:09,622 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-05 12:17:09,812 INFO mapreduce.Job: The url to track the job: http://localhost:8080/
2025-03-05 12:17:09,814 INFO mapreduce.Job: Running job: job_local212957500_0001
2025-03-05 12:17:09,825 INFO mapred.LocalJobRunner: OutputCommitter set in config null
2025-03-0

15. Compruebo el directorio de salida `apart_14`.

In [12]:
!ls -l /content/apart_14/

total 4
-rw-r--r-- 1 root root 72 Mar  5 12:17 part-00000
-rw-r--r-- 1 root root  0 Mar  5 12:17 _SUCCESS


#VII: Streaming con Python sin Reductor

En este caso vamos a usar un programa en Python que recupere de forma aleatoria parte de las filas del fichero.

El código siguiente **NO es necesario ejecutarlo**, lo tienes en los recursos de *Aules* del ejercicio práctico.con el nombre **r.py**. Simplemente súbelo dentro de la carpeta `/content/` utilzando la opción de *Subir* en el menú `Archivos`.

In [13]:
#!/usr/bin/python3
# -*- coding: utf-8 -*-

import sys
import random
for line in sys.stdin:
   if (random.randint(1,100) <= int(sys.argv[1])):
           print(line.strip())

16. Queremos generar un **Map Reduce** con las especificaciones que aparecen a continuación:
- El fichero de **entrada** será `cite75_99.txt`
- Descargaremos la **salida** en la ruta `/content/apart_16`.
- Usaremos el fichero `r.py` para **mapear**, como tenemos que pasarle al sistema el valor de referencia (entre 1 y 100) para decidir si muestra cada línea, invocaremos al script con la siguiente sintaxis `"python r.py 50"`, indicando el 50 que deseamos seleccionar aproximadamente el 50% de las líneas del fichero de origen.
- No usaremos **reductor**.


**ATENCIÓN:** Recuerda que tienes que dar permisos de ejecución al script `r.py` para que *Hadoop Streaming* lo pueda procesar.

In [50]:
!chmod +x r.py

In [52]:
!mapred streaming -input /content/cite75_99.txt -output /content/apart_16 -mapper "python /content/r.py 50" -file /content/r.py

2025-03-05 15:26:43,716 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/r.py] [] /tmp/streamjob10911387047402369761.jar tmpDir=null
2025-03-05 15:26:44,439 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-05 15:26:44,618 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-05 15:26:44,618 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-05 15:26:44,663 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-05 15:26:44,903 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-05 15:26:44,929 INFO mapreduce.JobSubmitter: number of splits:8
2025-03-05 15:26:45,234 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local773179191_0001
2025-03-05 15:26:45,234 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-05 15:26:45,749 INFO mapred.LocalDistributedCacheManager:

17. Muestra el contenido de la carpeta `/content/apart_16` con el modificador `-l` para poder observar el tamaño de los elementos contenidos en la misma.

In [53]:
!ls -l /content/apart_16/

total 136976
-rw-r--r-- 1 root root 140259894 Mar  5 15:27 part-00000
-rw-r--r-- 1 root root         0 Mar  5 15:27 _SUCCESS


18. Vuelve a crear un **Map Reduce** similar al del apartado 16 pero ahora quiero que seleccione como máximo un 25% de las líneas del fichero de origen. La carpeta de salida será `/content/apart_18`

In [54]:
!mapred streaming -input /content/cite75_99.txt -output /content/apart_18 -mapper "python /content/r.py 25" -file /content/r.py

2025-03-05 15:28:13,722 WARN streaming.StreamJob: -file option is deprecated, please use generic option -files instead.
packageJobJar: [/content/r.py] [] /tmp/streamjob7690562493462414160.jar tmpDir=null
2025-03-05 15:28:14,470 INFO impl.MetricsConfig: Loaded properties from hadoop-metrics2.properties
2025-03-05 15:28:14,669 INFO impl.MetricsSystemImpl: Scheduled Metric snapshot period at 10 second(s).
2025-03-05 15:28:14,669 INFO impl.MetricsSystemImpl: JobTracker metrics system started
2025-03-05 15:28:14,692 WARN impl.MetricsSystemImpl: JobTracker metrics system already initialized!
2025-03-05 15:28:14,974 INFO mapred.FileInputFormat: Total input files to process : 1
2025-03-05 15:28:15,001 INFO mapreduce.JobSubmitter: number of splits:8
2025-03-05 15:28:15,303 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_local1227865315_0001
2025-03-05 15:28:15,303 INFO mapreduce.JobSubmitter: Executing with tokens: []
2025-03-05 15:28:15,739 INFO mapred.LocalDistributedCacheManager:

19. Muestra el contenido de la carpeta `/content/apart_18` con el modificador `-l` para poder observar el tamaño de los elementos contenidos en la misma.

In [55]:
!ls -l /content/apart_18/

total 68544
-rw-r--r-- 1 root root 70187660 Mar  5 15:28 part-00000
-rw-r--r-- 1 root root        0 Mar  5 15:28 _SUCCESS


20. Escribe en el cuadro de texto que tienes a continuación el tamaño que tiene cada fichero `part-00000` obtenidos de la ejecución de los procesos **Map Reduce** de los aprtados 16 y 18.
- ¿La diferencia de tamaño corresponde aproximadamente con el porcentaje de valor pasado como parámetro al script `r.py`?

*   Tamaño part-00000 del apartado 16 (50%): 140259894  bytes
*   Tamaño part-00000 del apartado 18 (25%): 70187660   bytes

Sabiendo que el tamaño original del fichero es de 280,519,788 bytes diría que sí corresponde la diferencia de tamaño apróximadamente con el porcentaje de cada uno de ellos.
